In [1]:
import pandas as pd 
import numpy as np

In [2]:
test_info = pd.read_csv('data/test.csv', usecols=['item_id', 'user_id'])

In [3]:
submission = pd.read_csv('stack_test.csv')
submission.head(3)

,item_id,deal_probability
0,6544e41a8817,0.348499
1,65b9484d670f,0.173824
2,8bab230b2ecd,0.170678


In [4]:
leak = pd.read_csv('user_id_leak.csv')
leak_user_id = leak.user_id
leak = leak.set_index('user_id')
leak.head(3)

,user_counts,avg_dp,avg_itsn,test_usr_cnts
user_id,,,,
41f112adb74b,512,0.0,670.843750,35
9bdc2002bf51,360,0.0,528.327778,16
f33560ee3a3d,289,0.0,615.705882,17


In [5]:
submission['leak_dp'] = test_info['user_id'].map(leak.avg_dp)
submission.head(3)

,item_id,deal_probability,leak_dp
0,6544e41a8817,0.348499,NaN
1,65b9484d670f,0.173824,NaN
2,8bab230b2ecd,0.170678,NaN


In [6]:
test_info['user_id'].isin(leak_user_id).value_counts()

False    441054
True      67384
Name: user_id, dtype: int64

In [7]:
submission['final_deal_probability'] = np.where(submission['leak_dp'].isnull(), 
                                                submission['deal_probability'], 
                                                submission['leak_dp'])
submission.head(3)

,item_id,deal_probability,leak_dp,final_deal_probability
0,6544e41a8817,0.348499,NaN,0.348499
1,65b9484d670f,0.173824,NaN,0.173824
2,8bab230b2ecd,0.170678,NaN,0.170678


In [8]:
(submission['deal_probability']-submission['final_deal_probability']).sum()

6179.3916544230087

In [9]:
submission['deal_probability']=submission['final_deal_probability']
submission[['item_id', 'deal_probability']].to_csv('leak_stack_test.csv', index=False)

In [13]:
submission[['item_id', 'deal_probability']].iloc[453567:,].shape

(54871, 2)